In [13]:
import joblib
import function_library as fl
import importlib
import pandas as pd
importlib.reload(fl)

<module 'function_library' from 'C:\\Users\\leere\\PycharmProjects\\Football_ML3\\function_library.py'>

In [14]:
file_path = r"C:\Users\leere\OneDrive\Desktop\RAW DATA\GOALS_LIVE.CSV"

# List of common encodings to try
encodings = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252']

data = None
for encoding in encodings:
    try:
        data = pd.read_csv(file_path, encoding=encoding)
        print(f"Successfully read the file with encoding: {encoding}")
        break
    except UnicodeDecodeError:
        print(f"Failed to decode with encoding: {encoding}")

col_dict = {
    "country": "Country",
    "league": "League",
    "datameci": "Date",
    "etapa": "Round",
    "txtechipa1": "home_team",
    "txtechipa2": "away_team",
    "place1t": "Home_team_place_total",
    "place1a": "Home_team_place_home",
    "place2t": "Away_team_place_total",
    "place2d": "Away_team_place_away",
    "customh": "ELO_home",
    "customa": "ELO_away",
    "custom3": "FORM_home",
    "custom4": "FORM_away",
    "home_val": "home_win",
    "home_val_2": "home_win_15",
    "home_val_3": "home_o25",
    "home_val_4": "home_o35",
    "home_val_5": "home_scored",
    "away_val": "away_win",
    "away_val_2": "away_win_15",
    "away_val_3": "away_o25",
    "away_val_4": "away_o35",
    "away_val_5": "away_scored",
    "scor1": "home_goals",
    "scor2": "away_goals",
    "cotao": "o2.5_odds",
}

data = data.rename(columns=col_dict).filter(items=col_dict.values())
# Convert Date column to datetime type
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')

# Order by date
data = data.sort_values(by='Date')

data = data[data["Round"] >= 8]
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

cols_to_drop = ['Date','home_team', 'away_team', 'home_goals', 'away_goals', 'o2.5_odds']

data_ready =data.drop(columns=cols_to_drop)
data_ready

Successfully read the file with encoding: utf-8


,Country,League,Round,Home_team_place_total,Home_team_place_home,Away_team_place_total,Away_team_place_away,ELO_home,ELO_away,FORM_home,...,home_win,home_win_15,home_o25,home_o35,home_scored,away_win,away_win_15,away_o25,away_o35,away_scored
0,England,Premier L,24,7,9,14,13,1822.0,1651.0,-3.0,...,45.455,27.273,63.636,27.273,1.818,27.273,18.182,36.364,27.273,1.091
1,Turkey,Super Lig,22,9,6,1,1,1918.0,2382.0,-2.0,...,60.000,30.000,30.000,10.000,1.300,88.889,44.444,77.778,44.444,2.778
2,Romania,Liga 1,24,12,12,7,6,1892.0,1946.0,1.0,...,36.364,9.091,36.364,18.182,1.182,27.273,9.091,36.364,27.273,1.000
3,Portugal,Primeira L,20,11,9,3,5,1803.0,2038.0,4.0,...,44.444,0.000,66.667,44.444,1.556,44.444,44.444,55.556,33.333,1.778
4,Portugal,Primeira L,20,10,8,4,2,1781.0,1998.0,-11.0,...,44.444,11.111,44.444,22.222,1.222,77.778,22.222,44.444,0.000,1.444
5,Poland,Ekstraklasa,19,18,18,11,9,1767.0,1874.0,-16.0,...,11.111,0.000,33.333,11.111,0.889,33.333,11.111,11.111,0.000,0.778
6,Netherlands,Eerste Divisie,24,20,19,17,16,1191.0,1377.0,13.0,...,16.667,8.333,58.333,41.667,0.917,16.667,8.333,75.000,25.000,1.167
7,Netherlands,Eerste Divisie,24,14,15,15,9,1415.0,1409.0,13.0,...,27.273,18.182,81.818,72.727,2.273,45.455,27.273,72.727,54.545,1.455
8,Netherlands,Eerste Divisie,21,6,4,7,15,1503.0,1530.0,-8.0,...,72.727,63.636,81.818,81.818,2.818,20.000,20.000,60.000,40.000,1.700
9,Israel,Premier League,21,2,4,3,4,2255.0,2071.0,0.0,...,66.667,33.333,55.556,44.444,2.222,50.000,40.000,80.000,50.000,2.000


In [15]:
# Directory where models are saved
models_dir = "ALL_MODELS"

# Group unseen data by League and Sub-League
grouped_data = data_ready.groupby(['Country', 'League'])

# Dictionary to store predictions
predictions = {}

# Iterate through each group in the unseen data
for (country, league), group in grouped_data:
    # Construct the model filename
    model_filename = f"{models_dir}/{country}_{league}_model.pkl"

    try:
        # Load the corresponding model
        saved_data = joblib.load(model_filename)  # Load the entire dictionary
        model = saved_data['model']  # Extract the model
        scaler = saved_data['scaler']  # Extract the scaler
        threshold = saved_data['threshold']  # Extract the threshold

        # Extract features for prediction (exclude grouping columns)
        X = group.drop(columns=['Country', 'League'])

        # Scale the features
        X_scaled = scaler.transform(X)

        # Make probability predictions using the model
        y_proba = model.predict_proba(X_scaled)[:, 1]  # Positive class probabilities

        # Filter predictions based on the threshold
        group['Probability'] = y_proba
        group['Prediction'] = (y_proba >= threshold).astype(int)

        # Keep only rows where Prediction == 1
        filtered_predictions = group[group['Prediction'] == 1]

        # Add `home_team` and `away_team` from the original `data`
        game_info = data.loc[filtered_predictions.index, ['home_team', 'away_team', 'Date']]
        filtered_predictions = pd.concat([game_info, filtered_predictions], axis=1)

        # Add filtered predictions to the dictionary
        predictions[(country, league)] = filtered_predictions
        print(f"Filtered predictions made for {country} - {league}")

    except FileNotFoundError:
        print(f"Model not found for {country} - {league}. Skipping...")

    except Exception as e:
        print(f"Error processing {country} - {league}: {e}")

# Combine all predictions into a single DataFrame
if predictions:
    all_predictions = pd.concat(predictions.values(), ignore_index=True)
    print("\nAll Predictions as a DataFrame:")
    print(all_predictions)
else:
    print("No predictions available.")


Model not found for Australia - A-League. Skipping...
Filtered predictions made for Czech - Division 1
Filtered predictions made for England - Championship
Filtered predictions made for England - League Two
Filtered predictions made for England - Premier L
Filtered predictions made for Greece - Super L
Filtered predictions made for Israel - Premier League
Filtered predictions made for Italy - Serie A
Filtered predictions made for Netherlands - Eerste Divisie
Filtered predictions made for Poland - Ekstraklasa
Filtered predictions made for Portugal - Primeira L
Filtered predictions made for Romania - Liga 1
Filtered predictions made for Scotland - Premier L
Filtered predictions made for Spain - Primera
Filtered predictions made for Spain - Segunda
Model not found for Switzerland - Challenge Leagu. Skipping...
Filtered predictions made for Switzerland - Super League
Filtered predictions made for Turkey - Super Lig

All Predictions as a DataFrame:
           home_team         away_team    

In [16]:
#all_predictions

In [17]:
all_predictions = fl.team_name_map(all_predictions)
all_predictions

,home_team,away_team,Date,Country,League,Round,Home_team_place_total,Home_team_place_home,Away_team_place_total,Away_team_place_away,...,home_o25,home_o35,home_scored,away_win,away_win_15,away_o25,away_o35,away_scored,Probability,Prediction
0,Burnley,Oxford Utd,2025-02-04,England,Championship,31,3,8,16,20,...,21.429,7.143,1.071,7.143,0.000,42.857,21.429,0.714,0.546679,1
1,Asteras Tripolis,Lamia,2025-02-03,Greece,Super L,21,5,5,14,13,...,40.000,0.000,1.200,10.000,0.000,30.000,20.000,0.700,0.545455,1
2,Maccabi Tel Aviv,Beitar Jerusalem,2025-02-03,Israel,Premier League,21,2,4,3,4,...,55.556,44.444,2.222,50.000,40.000,80.000,50.000,2.000,0.561796,1
3,Cagliari,Lazio,2025-02-03,Italy,Serie A,23,16,17,4,6,...,45.455,36.364,1.091,54.545,18.182,63.636,27.273,1.636,0.688205,1
4,Jong Alkmaar,VVV Venlo,2025-02-03,Netherlands,Eerste Divisie,24,14,15,15,9,...,81.818,72.727,2.273,45.455,27.273,72.727,54.545,1.455,0.866667,1
5,Wroclaw,Gliwice,2025-02-03,Poland,Ekstraklasa,19,18,18,11,9,...,33.333,11.111,0.889,33.333,11.111,11.111,0.000,0.778,0.650000,1
6,Rapid Bucharest,Unirea Slobozia,2025-02-04,Romania,Liga 1,25,6,2,13,9,...,33.333,25.000,1.583,33.333,8.333,50.000,8.333,0.833,0.578337,1
7,Sepsi,Constanta,2025-02-06,Romania,Liga 1,25,8,6,12,13,...,16.667,0.000,1.167,8.333,0.000,33.333,33.333,0.750,0.592478,1
8,Celtic,Dundee,2025-02-05,Scotland,Premier L,24,1,2,10,8,...,72.727,45.455,3.091,27.273,9.091,45.455,36.364,1.091,0.647669,1
9,Girona,Las Palmas,2025-02-03,Spain,Primera,22,8,10,15,12,...,80.000,40.000,1.900,30.000,10.000,70.000,40.000,1.200,0.540850,1


In [18]:
fl.create_import_file(all_predictions, r"C:\Users\leere\OneDrive\Desktop\IMPORTS\LAY_U25_ALL_IMPORT.csv", provider="lay_all_u25", market_name="Over/Under 2.5 Goals", selection_name="Under 2.5 Goals")

File created and saved successfully at: C:\Users\leere\OneDrive\Desktop\IMPORTS\LAY_U25_ALL_IMPORT.csv
